<a href="https://colab.research.google.com/github/chrispi21/python-dataeng/blob/main/06_programowanie_obiektowe_i_obsluga_wyjatkow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>